# Dataset:
You will be working with two datasets: Train and Test, both containing images of 40 individuals. Each dataset has been preprocessed for this assignment.

In [81]:
# import all libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA


# Create PCA function that takes two inputs: dataset, and k= number of principle component, and return the transform data

In [82]:
def my_pca(Data,k):
    "Build this function"
    mean_vector = np.mean(Data, axis=0)
    centered_data = Data - mean_vector

    # Step 2: Calculate the Covariance Matrix
    covariance_matrix = np.cov(centered_data, rowvar=False)

    # Step 3: Find Eigenvalues and Eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

    # Step 4: Sort Eigenvalues and Eigenvectors
    sorted_indices = np.argsort(eigenvalues)[::-1]
    eigenvalues = eigenvalues[sorted_indices]
    eigenvectors = eigenvectors[:, sorted_indices]

    # Step 5: Select the top k eigenvectors as principal components
    principal_components = eigenvectors[:, :k]

    # Step 6: Project the data onto the selected principal components
    X_reduced = np.dot(principal_components.transpose(),centered_data.transpose()).transpose()

#     projected_data = np.dot(centered_data, principal_components)

    return X_reduced


In [83]:
def my_pca(Data , k):
     
    #Step-1
    X_meaned = Data - np.mean(Data , axis = 0)
     
    #Step-2
    cov_mat = np.cov(X_meaned , rowvar = False)
     
    #Step-3
    eigen_values , eigen_vectors = np.linalg.eigh(cov_mat)
     
    #Step-4
    sorted_index = np.argsort(eigen_values)[::-1]
    sorted_eigenvalue = eigen_values[sorted_index]
    sorted_eigenvectors = eigen_vectors[:,sorted_index]
     
    #Step-5
    eigenvector_subset = sorted_eigenvectors[:,0:k]
     
    #Step-6
    X_reduced = np.dot(eigenvector_subset.transpose() , X_meaned.transpose() ).transpose()
     
    return X_reduced

# Read your TrainData and Test data. Try to remove the last column of the training data and assign it to one variable, do same thing for the TestData

In [84]:
# write the code here
df_train = pd.read_csv('TrainData.csv')
X_train = df_train.iloc[:,:-1]
X_test = df_train.iloc[:,-1]

In [85]:
df_test = pd.read_csv('TestData.csv')
y_train = df_test.iloc[:,:-1]
y_test = df_test.iloc[:,-1]

# Apply my_pca 

In [86]:
# write the code here
result = my_pca(X_train,2)

In [104]:
result_y = my_pca(y_train,4)

# Apply pca using sklearn

In [117]:
# write the code here
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sb
import matplotlib.pyplot as plt
scaling=StandardScaler()
Scaled_data=scaling.fit_transform(X_train)
Scaled_data_y = scaling.fit_transform(y_train)

principal=PCA(n_components=3)

x=principal.fit_transform(Scaled_data)
y = principal.fit_transform(Scaled_data_y)


# plt.figure(figsize = (6,6))
# sb.scatterplot(data = principal_df , x = 'PC1',y = 'PC2' , hue = 'T_10305' , s = 60 , palette= 'icefire')


# Create kernel PCA function that takes two inputs: dataset, and k= number of principle component, and return the transform data. In addition, you need to create three other function one for rbf_kernel, one for polynomial_kernel, and one for linear_kernel

In [89]:
import numpy as np

def rbf_kernel(x, y, gamma=1.0):
    return np.exp(-gamma * np.linalg.norm(x - y) ** 2)

def poly_kernel(x, y, degree=3):
    return (np.dot(x, y) + 1) ** degree

def linear_kernel(x, y):
    
    return np.dot(x, y)

def my_kpca(data, n_components, kernel_type='rbf', kernel_param=1.0):
    """
    Kernel Principal Component Analysis (KPCA) function.

    Parameters:
    - data: Input data as an ndarray of shape (n_samples, n_features).
    - n_components: Number of principal components to retain.
    - kernel_type: Type of kernel ('rbf', 'poly', or 'linear').
    - kernel_param: Kernel parameter (e.g., gamma for RBF, degree for polynomial).

    Returns:
    - Transformed data in the KPCA space.
    """
    
    n_samples, n_features = data.shape
    K = np.zeros((n_samples, n_samples))

    # Calculate the kernel matrix
    for i in range(n_samples):
        for j in range(n_samples):
            if kernel_type == 'rbf':
                K[i, j] = rbf_kernel(data[i], data[j], gamma=kernel_param)
            elif kernel_type == 'poly':
                K[i, j] = poly_kernel(data[i], data[j], degree=kernel_param)
            elif kernel_type == 'linear':
                K[i, j] = linear_kernel(data[i], data[j])

    # Center the kernel matrix
    one_n = np.ones((n_samples, n_samples)) / n_samples
    K_centered = K - np.dot(one_n, K) - np.dot(K, one_n) + np.dot(np.dot(one_n, K), one_n)

    # Compute eigenvalues and eigenvectors of the centered kernel matrix
    eigvals, eigvecs = np.linalg.eigh(K_centered)

    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eigvals)[::-1]
    eigvals = eigvals[sorted_indices]
    eigvecs = eigvecs[:, sorted_indices]

    # Select the top n_components eigenvectors
    eigvecs = eigvecs[:, :n_components]

    # Project the data into KPCA space
    kpca_data = np.dot(K_centered, eigvecs)

    return kpca_data



# Apply my_kpca on the Train data


In [90]:
X_train_kpca = my_kpca(train_x.to_numpy(),2)

# Apply Kpca using sklearn on Training data

In [91]:
# build your code here
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import KernelPCA
stndS = StandardScaler()
XTrain = stndS.fit_transform(train_x)
XTest = stndS.fit_transform(test_x)

Kernel_pca = KernelPCA(n_components = 2, kernel= "rbf")

In [92]:
# build your code her 
XTrain = Kernel_pca.fit_transform(XTrain)
XTest = Kernel_pca.transform(XTest)

# You can use the functions below as your classifier 


In [93]:
# Function to calculate distance between two points
def dis(x1, x2):
    return np.linalg.norm(x1 - x2)

# Function to perform classification 
def myclassifier(Train, Trainlabel, Test):
    " Train is the training data"
    " Trainlabel is the training labels"
    " Test is the testing data"
    pred = []

    for testpoint in Test:
        pred_dis = []
        for trainpoint in Train:
            pred_dis.append(dis(testpoint, trainpoint))

        pred.append(np.argmin(pred_dis))  

    return np.array(pred)


# Below function is to calculte the accuracy , you can use this function to get the accuracy of pca and kpca

In [94]:
def calculate_accuracy(true_labels, predicted_labels):
    # Ensure that the true labels and predicted labels have the same length
    if len(true_labels) != len(predicted_labels):
        raise ValueError("Length of true_labels and predicted_labels must be the same.")

    # Count the number of correct predictions
    correct_predictions = sum(1 for true, predicted in zip(true_labels, predicted_labels) if true == predicted)

    # Calculate accuracy as the ratio of correct predictions to total predictions
    accuracy = correct_predictions / len(true_labels)

    return accuracy


In [118]:
pred = myclassifier(x,X_test,y)

In [119]:
calculate_accuracy(y_test,pred)

0.01

# PCA summary

The provided code is an implementation of Principal Component Analysis (PCA) from scratch. PCA is a dimensionality reduction technique used to reduce the number of features (variables) in a dataset while preserving as much of the original information as possible. Here's a summary of each step in the code:

1. **Mean Centering:**
   - `X_meaned = Data - np.mean(Data, axis=0)` subtracts the mean of each feature (column) from the data points. This centers the data around the origin.

2. **Covariance Matrix Calculation:**
   - `cov_mat = np.cov(X_meaned, rowvar=False)` computes the covariance matrix of the mean-centered data. The `rowvar=False` argument indicates that each column represents a variable, not a row.

3. **Eigenvalue and Eigenvector Calculation:**
   - `eigen_values, eigen_vectors = np.linalg.eigh(cov_mat)` calculates the eigenvalues and eigenvectors of the covariance matrix. These eigenvalues and eigenvectors represent the directions and magnitudes of maximum variance in the data.

4. **Sorting Eigenvalues and Eigenvectors:**
   - `sorted_index = np.argsort(eigen_values)[::-1]` sorts the eigenvalues in descending order, and `sorted_eigenvalue` and `sorted_eigenvectors` store them accordingly. This step ensures that the principal components are ranked by their importance.

5. **Selecting the Top k Eigenvectors:**
   - `eigenvector_subset = sorted_eigenvectors[:, 0:k]` selects the top k eigenvectors, corresponding to the k largest eigenvalues. These eigenvectors will be used as the principal components.

6. **Projection:**
   - `X_reduced = np.dot(eigenvector_subset.transpose(), X_meaned.transpose()).transpose()` projects the mean-centered data onto the selected principal components. This step reduces the dimensionality of the data while retaining most of the original information.

Overall, this code effectively performs PCA and returns the dataset reduced to k dimensions. You can use this function to apply PCA to your data when you specify the input data (`Data`) and the desired number of principal components (`k`).

# Choose an appropriate number of principal component

To choose an appropriate number of principal components to retain a significant amount of variance in your dataset, you can follow these steps:

1. **Perform PCA:** Apply Principal Component Analysis (PCA) to your dataset. This involves mean-centering your data, calculating the covariance matrix, and computing the eigenvalues and eigenvectors of the covariance matrix.

2. **Sort Eigenvalues:** Sort the eigenvalues in descending order. Eigenvalues represent the amount of variance explained by each principal component, with larger eigenvalues explaining more variance.

3. **Calculate Explained Variance Ratio:** Calculate the explained variance ratio for each principal component. This is done by dividing each eigenvalue by the sum of all eigenvalues. It represents the proportion of total variance explained by that component.

4. **Cumulative Explained Variance:** Calculate the cumulative explained variance by summing the explained variance ratios from the first principal component up to the current component. This tells you how much of the total variance is explained as you add more components.

5. **Set a Threshold:** Decide on a threshold for the amount of variance you want to retain. For example, you mentioned retaining a "significant" amount, which might be 95% of the variance.

6. **Select k Components:** Choose the number of principal components (k) such that the cumulative explained variance exceeds or reaches your chosen threshold. This ensures that you retain a significant amount of variance while reducing the dimensionality of the data.

7. **Apply PCA with Selected k:** Finally, apply PCA again, but this time, only retain the top k principal components. These k components capture the most important information in your data while reducing its dimensionality.

For example, if you find that the cumulative explained variance reaches or exceeds 95% when you include the first 10 principal components, you can choose k = 10 as an appropriate number of components to retain a significant amount of variance.

Keep in mind that the appropriate value of k may vary depending on your specific dataset and the trade-off between dimensionality reduction and retaining information. You can adjust the threshold to meet your specific needs, such as 90%, 95%, or 99% explained variance.

To retain the significant value of variance upto 95%, value of principal component should be 5